In [43]:
import pennylane as qml
from matplotlib import pyplot as plt
import numpy as np
from numpy import array
import scipy
from scipy.optimize import minimize
import networkx as nx
import itertools

np.random.seed(42)

Defining the hamiltonian for the graph where we have qubits as part of a cyclic graph, the heisenberg hamiltonian is given as:
$$\hat{H} = \sum_{(i,j)\in E} X_iX_j + Y_iY_j + Z_iZ_j$$ 
where we have X, Y and Z as the pauli matrices acting on the i-th qubit. E is the set of the edges, along which the qubits are interacting, of the cyclic graph.

In [23]:
def create_hamiltonian_matrix(n, graph):

    matrix = np.zeros((2 ** n, 2 ** n))

    for i in graph.edges:
        x = y = z = 1
        for j in range(0, n):
            if j == i[0] or j == i[1]:
            # If an edge exists between the qubit connection
                x = np.kron(x, qml.PauliX.matrix)
                y = np.kron(y, qml.PauliY.matrix)
                z = np.kron(z, qml.PauliZ.matrix)
            else:
            # If there is not edge(interaction) between the qubits
                x = np.kron(x, np.identity(2))
                y = np.kron(y, np.identity(2))
                z = np.kron(z, np.identity(2))
        matrix = np.add(matrix, np.add(x, np.add(y, z)))

    return matrix
interaction_graph = nx.cycle_graph(4)
ham_matrix = create_hamiltonian_matrix(4, interaction_graph)


# VQT

Giving the initial parameters $\beta$, which is the inverse temperature for which we need to simulate the thermal state for the given hamiltonian.
$$ \hat{\sigma}_\beta = \frac{1}{Z_\beta}e^{-\beta \hat{H}}, Z_\beta=tr(e^{-\beta\hat{H}})$$ 

In [24]:
beta = 2
# Number of qubits for the circuit will be 4, since we are simulating a 4 qubit hamiltonian.
num_qubits = 4

## Iniitialisation

The first step of initialisation is the create an initial density matrix of the state which will be stored classically, $p_\theta$. We consider $p_\theta$ distribution to be factorized latent space model which means that we can write it as an uncorrelated tensor product of 4 one-qubit density matrix that are diagonal in computational basis. Its major motivation is that $p_\theta$ scales linearly with number of qubits rather than scaling exponentially.
$$p_\theta^i = p_i(\theta_i)\ket{0}\bra{0} + (1-p_i(\theta_i))\ket{1}\bra{1} $$ 
we can define $p_i(\theta_i)$ to be sigmoid.
$$ p_i(\theta_i) = \frac{e^{\theta_i}}{e^{\theta_i}+1} $$

In [25]:
def prob_distribution(params):
    sigmoid = np.exp(params)/(np.exp(params)+1)
    return np.vstack([sigmoid, 1-sigmoid]).T

## Ansatz Circuit

The ansatz circuit or the $U(\phi)$ which is the quantum computation part of the algorithm. It is comprised of the rotation and coupling layers. These rotations and coupling parameters are $\phi$ which we manipulate during the process of the optimisation.
To contruct the ansatz we combine the paramerterized rotation and coupling gates. The rotation layer is simply $R_x$ $R_y$ $R_z$ gates applied to each qubits. The coupling layer is coupling gates placed between teh edge of the interaction graph.

In [42]:
# defining the depth of the circuit
depth = 4
device = qml.device("default.qubit", wires = num_qubits)
def ansatz(rotation_parameters, coupling_parameters, sample):
    # Preparing the initial qubit basis state according to the sample
    qml.BasisStatePreparation(sample, wires=range(num_qubits))

    for i in range(0, depth):
        rotations = ["Z", "X", "Y"]
        # Rotation layer
        for j in range(len(rotations)):
            qml.AngleEmbedding(rotation_parameters[i][j], wires=range(num_qubits), rotation=rotations[j])
        # Coupling layer (doing controlled rotation action on the qubits that have to coupled)
        qml.broadcast(
            unitary=qml.CRX,
            pattern="ring",
            wires=range(num_qubits),
            parameters=coupling_parameters[i]
        )
    # doing measurement on the circuit
    return qml.expval(qml.Hermitian(ham_matrix, wires=range(num_qubits)))

# Constructing the qnode for the ansatz circuit
qnode = qml.QNode(ansatz, device)
rotation_params = [[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]] for i in range(0, depth)]
coupling_params = [[1, 1, 1, 1] for i in range(0, depth)]
print(qml.draw(qnode, expansion_strategy="device")(rotation_params, coupling_params, sample=[0, 1, 0, 1]))

 0: ──RZ(1)──RX(1)──RY(1)─────────╭C─────────────────────────────╭RX(1)──RZ(1)──RX(1)──RY(1)──╭C─────────────────────────────╭RX(1)──RZ(1)──RX(1)──RY(1)──╭C─────────────────────────────╭RX(1)──RZ(1)──RX(1)──RY(1)──╭C──────────────────────╭RX(1)──╭┤ ⟨H0⟩ 
 1: ──X──────RZ(1)──RX(1)──RY(1)──╰RX(1)──╭C───────RZ(1)──RX(1)──│───────RY(1)────────────────╰RX(1)──╭C───────RZ(1)──RX(1)──│───────RY(1)────────────────╰RX(1)──╭C───────RZ(1)──RX(1)──│───────RY(1)────────────────╰RX(1)──╭C──────────────│───────├┤ ⟨H0⟩ 
 2: ──RZ(1)──RX(1)──RY(1)─────────────────╰RX(1)──╭C──────RZ(1)──│───────RX(1)──RY(1)─────────────────╰RX(1)──╭C──────RZ(1)──│───────RX(1)──RY(1)─────────────────╰RX(1)──╭C──────RZ(1)──│───────RX(1)──RY(1)─────────────────╰RX(1)──╭C──────│───────├┤ ⟨H0⟩ 
 3: ──X──────RZ(1)──RX(1)──RY(1)──────────────────╰RX(1)─────────╰C──────RZ(1)──RX(1)──RY(1)──────────────────╰RX(1)─────────╰C──────RZ(1)──RX(1)──RY(1)──────────────────╰RX(1)─────────╰C──────RZ(1)──RX(1)──RY(1)──────────────────╰RX(1

## Entropy

The von neumann entropy of the state, which is determined by collection of $p_{\theta_i}$. Since the entropu of a collection of multiple uncorrelated subsystem, we can sum the entropy for each subsystem, we can sum the entropy values of each one-qubit system in teh factorized space to get the total, the Von Neumann entropy of the latent and visible states are also identical due to the invariance of Von Neumann entropy under unitary action.

In [35]:
def entropy(distribution):
    total_entropy = 0
    for d in distribution:
        total_entropy += -1*d[0]*np.log(d[0]) + -1*d[1]*np.log(d[1])
    return total_entropy

## The Cost Function

We combine the ansatz and the entropy to get the cost function, we use ansatz to calculate $\bra{x_i}U^{\dagger}(\phi)\hat{H}U(\phi)\ket{x_i}$ for each basis state $\ket{x_i}$. We then multiply these expectation values by their corresponding $(p_\theta)_i$, which is exactly the probability of sampling $\ket{x_i}$ from the distribution. Summing these gives us the expected value of the hamiltonian.

$$ L_{VQT}(\theta,\phi) = \beta tr(\hat{p}_{\theta\phi}\hat{H}) - S(\hat{p}_{\theta\phi}) $$

In [33]:
def exact_cost(params):

    # Arranging the parameters
    dist_params = params[0:num_qubits]
    ansatz_params_1 = params[num_qubits : ((depth + 1) * num_qubits)]
    ansatz_params_2 = params[((depth + 1) * num_qubits) :]

    coupling = np.split(ansatz_params_1, depth)

    # Partitions the parameters into multiple lists
    split = np.split(ansatz_params_2, depth)
    rotation = []
    for s in split:
        rotation.append(np.split(s, 3))

    ansatz_params = [rotation, coupling]
    # The probability distribution of the initial state
    distribution = prob_distribution(dist_params)
    combos = itertools.product([0, 1], repeat=num_qubits)
    set = [list(c) for c in combos]
    cost = 0
    # The value of the hamiltonian for the distribution using the quantum ansatz circuit
    for i in set:
        quant_result = qnode(ansatz_params[0], ansatz_params[1], sample=i)
        for j in range(0, len(i)):
            quant_result = quant_result*distribution[j][i[j]]
        cost += quant_result
    # The entropy of the system
    entropy_cost = entropy(distribution)
    # The final cost function, the formula for which is defined above
    final_cost = beta*cost - entropy_cost

    return final_cost

## Optimisation Function

In [29]:
def cost_optimisation(params):
    global iterations
    cost = exact_cost(params)
    if iterations % 50 == 0:
        print("Cost at Step {}: {}".format(iterations, cost))

    iterations += 1
    return cost

Using the COBYLA optimizer to minimise the cost function that we defined.

Also, we are generating random parameters for the initialisation of the circuit.

In [36]:
iterations = 0

number = num_qubits * (1 + depth * 4)
params = [np.random.randint(-200, 200) / 100 for i in range(0, number)]
out = minimize(cost_optimisation, x0=params, method="COBYLA", options={"maxiter": 1600})
out_params = out["x"]

Cost at Step 0: -2.2708453033289824
Cost at Step 50: -3.5526991851098946
Cost at Step 100: -4.985514622752113
Cost at Step 150: -6.786081932766198
Cost at Step 200: -7.020076718646381
Cost at Step 250: -7.780378451484982
Cost at Step 300: -8.798373940764044
Cost at Step 350: -10.1665993471928
Cost at Step 400: -10.886237535082678
Cost at Step 450: -11.144180330356843
Cost at Step 500: -11.21272487807346
Cost at Step 550: -11.71773889977797
Cost at Step 600: -12.475937469065913
Cost at Step 650: -12.99805668715278
Cost at Step 700: -13.23280893979654
Cost at Step 750: -13.424297883089249
Cost at Step 800: -13.535958968766002
Cost at Step 850: -13.736168688024165
Cost at Step 900: -13.863802393772236
Cost at Step 950: -14.087521403610259
Cost at Step 1000: -14.03504954673694
Cost at Step 1050: -14.311088524280688
Cost at Step 1100: -14.343668942931105
Cost at Step 1150: -14.369165447851893
Cost at Step 1200: -14.478481973785733
Cost at Step 1250: -14.60599355285146
Cost at Step 1300: -14

## Verifying Results

Now we have the final optimized parameters, to compare the results we construct a density matrix, from the given vaues of $\theta$ and $\phi$ from some initial state.

In [37]:
def prepare_state(params, device):
    density_matrix_test = np.zeros((2 ** num_qubits, 2 ** num_qubits))
    dist_params = params[0:num_qubits]
    ansatz_params_1 = params[num_qubits : ((depth + 1) * num_qubits)]
    ansatz_params_2 = params[((depth + 1) * num_qubits) :]

    coupling = np.split(ansatz_params_1, depth)

    # Partitions the parameters into multiple lists
    split = np.split(ansatz_params_2, depth)
    rotation = []
    for s in split:
        rotation.append(np.split(s, 3))

    ansatz_params = [rotation, coupling]
    distribution = prob_distribution(dist_params)
    combos = itertools.product([0, 1], repeat=num_qubits)
    set = [list(c) for c in combos]
    # Runs the circuit in the case of the optimal parameters, for each bitstring,
    # and adds the result to the final density matrix
    for i in set:
        qnode(ansatz_params[0], ansatz_params[1], sample=i)
        state = device.state
        for j in range(len(i)):
            state = np.sqrt(distribution[j][i[j]]) * state
        density_matrix_test = np.add(density_matrix_test, np.outer(state, np.conj(state)))

    return density_matrix_test 
#  Prepares the density matrix
density_matrix_test = prepare_state(out_params, device)
    

In [39]:
def create_target(qubit, beta, ham, graph):

    # Calculates the matrix form of the density matrix, by taking
    # the exponential of the Hamiltonian

    h = ham(qubit, graph)
    y = -1 * float(beta) * h
    new_matrix = scipy.linalg.expm(np.array(y))
    norm = np.trace(new_matrix)
    final_target = (1 / norm) * new_matrix

    return final_target


target_density_matrix = create_target(
    num_qubits, beta,
    create_hamiltonian_matrix,
    interaction_graph
    )

traceDistance = 0.5 * np.trace(np.absolute(np.add(target_density_matrix, -1 * density_matrix_test)))
print("Trace Distance: " + str(traceDistance))


Trace Distance: 0.06426760247477861


As we can see the trace distance between the target matrix and the initial matrix is close 0, hence we can say that the thermal state that VQT has generated for the corresponding hamiltonian H is good approximation